# Feature Track 3: Maintain a Conversation

# Setup

In [2]:
from loguru import logger
import sys


from conversational_toolkit.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from conversational_toolkit.llms.base import LLMMessage, Roles
from conversational_toolkit.agents.base import QueryWithContext

from sme_kt_zh_collaboration_rag.feature0_baseline_rag import (
    load_chunks,
    build_vector_store,
    build_llm,
    build_agent,
    SYSTEM_PROMPT,
    EMBEDDING_MODEL,
)

logger.remove()
logger.add(sys.stderr, level="WARNING")  # hides DEBUG

c:\Users\sieverin\SDSC\Code\sme-kt-zh-collaboration-rag\rag_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Consider using the pymupdf_layout package for a greatly improved page layout analysis.


1

In [3]:
chunks = load_chunks(5)

embedding_model = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)
vector_store = await build_vector_store(chunks, embedding_model, reset=False)
llm = build_llm("openai", model_name="gpt-4o-mini")

5


In [4]:
agent = build_agent(
    vector_store,
    embedding_model,
    llm,
    top_k=5,
    system_prompt=SYSTEM_PROMPT,
)

# Iteration

At each step, the current query, and the history is sent to the LLM for it to answer based on that. This can induce a big conversation, which leads to cost and potentially loose of performance. Here the previous sources are not sent, but that could be a (costly) option.

In [5]:
query = "Which pallets in our portfolio have a third-party verified EPD?"
query_with_context = QueryWithContext(query=query, history=[])

response = await agent.answer(query_with_context)

print("Answer:")
print(f"{response.content}")
print(f"Sources ({len(response.sources)}):")
for src in response.sources:
    source_file = src.metadata.get("source_file", "?")
    print(f"  {source_file!r}  |  {src.title!r}")

Answer:
Based on the provided excerpts, I can confirm that there are products in PrimePack AG's portfolio that have third-party verified Environmental Product Declarations (EPDs). However, the specific products with verified EPDs are not listed in the excerpts provided.

The excerpts mention that independent third-party verification of declarations and data is conducted according to ISO 14025:2006, and that Bureau Veritas Italia S.p.A. is an approved certification body responsible for this verification (source: 9fe53c3c-32c8-4b4e-8f37-2ad9261fd30e). Additionally, the EPDs are subject to follow-up data verification during their validity (source: 4a540a55-d291-4fc3-a4e0-f60219ae5c50 and e5a54fa0-533b-4e45-9a1d-9da3c9585af1).

For a complete list of which specific pallets have third-party verified EPDs, you may need to refer to additional internal documentation or the product catalog, as the excerpts do not specify the names of those products.
Sources (5):
  'EPD_pallet_relicyc_logypal1.p

In [6]:
query_l = [
    "Which pallets in our portfolio have a third-party verified EPD?",
    "Rewrite what you just said as a pirate and very concisely.",
    "Make minutes of our conversation so far, you are 'pAIrate' and I'm AIvan",
]

history: list[LLMMessage] = []

for query in query_l:
    response = await agent.answer(QueryWithContext(query=query, history=history))

    print("Answer:")
    print(f"{response.content}")
    print(f"\nSources ({len(response.sources)}):")
    for src in response.sources:
        source_file = src.metadata.get("source_file", "?")
        print(f"  {source_file!r}  |  {src.title!r}")

    history.append(LLMMessage(content=query, role=Roles.USER))
    history.append(LLMMessage(content=response.content, role=Roles.ASSISTANT))

    print("-" * 80)

Answer:
Based on the provided excerpts, I can confirm that PrimePack AG has the following pallets in its portfolio that have third-party verified Environmental Product Declarations (EPDs):

1. **Noé Pallet (32-100, CPR System)**
2. **Wooden Pallet 1208 (32-101, CPR System)**
3. **Recycled Plastic Pallet (32-102, CPR System)**
4. **Logypal 1 (32-103, Relicyc)**
5. **LogyLight (32-104, Relicyc)**
6. **EP 08 (32-105, StabilPlastik)**

The third-party verification of the EPDs is conducted according to ISO 14025:2006 by accredited certification bodies such as Bureau Veritas Italia S.p.A. and WAP Sustainability Consulting (source: excerpts from various documents). 

If you need more specific details about each pallet's EPD, please let me know!

Sources (5):
  'EPD_pallet_relicyc_logypal1.pdf'  |  '### THIRD-PARTY VERIFICATION'
  'ART_response_inquiry_frische_felder.md'  |  '### Response to Section 3: General'
  'EVALUATION_qa_ground_truth.md'  |  '## Q1: portfolio_scope'
  'EPD_tape_IPG_hotm

In [7]:
history

[LLMMessage(content='Which pallets in our portfolio have a third-party verified EPD?', role=<Roles.USER: 'user'>, tool_calls=None, tool_call_id=None, name=None),
 LLMMessage(content="Based on the provided excerpts, I can confirm that PrimePack AG has the following pallets in its portfolio that have third-party verified Environmental Product Declarations (EPDs):\n\n1. **Noé Pallet (32-100, CPR System)**\n2. **Wooden Pallet 1208 (32-101, CPR System)**\n3. **Recycled Plastic Pallet (32-102, CPR System)**\n4. **Logypal 1 (32-103, Relicyc)**\n5. **LogyLight (32-104, Relicyc)**\n6. **EP 08 (32-105, StabilPlastik)**\n\nThe third-party verification of the EPDs is conducted according to ISO 14025:2006 by accredited certification bodies such as Bureau Veritas Italia S.p.A. and WAP Sustainability Consulting (source: excerpts from various documents). \n\nIf you need more specific details about each pallet's EPD, please let me know!", role=<Roles.ASSISTANT: 'assistant'>, tool_calls=None, tool_call_

# History summarization

One solution is to ask an LLM to summarize the conversation before each query and only send this summary to the LLM, potentially asking for a summary w.r.t. query. Depending on the implementation, it can reduce cost or improve performance. But one should be careful, as some relevant information might be lost.

In [8]:
async def summarize_history(history: list[LLMMessage]) -> str:
    llm_summarization = build_llm("openai", model_name="gpt-4o-mini")

    summarization_system_prompt = "You are a helpful assistant that summarizes conversations between a user and an assistant. Do not answer any question, just summarize the conversation in a concise way. The user is called 'User' and the assistant is called 'Assistant'."
    messages: list[LLMMessage] = [
        LLMMessage(content=summarization_system_prompt, role=Roles.SYSTEM),
        *history,
    ]

    msg = await llm_summarization.generate(conversation=messages)
    return msg.content

In [9]:
query_l = [
    "Which pallets in our portfolio have a third-party verified EPD?",
    "Rewrite what you just said as a pirate and very concisely.",
    "What is the current customer request?",
]

history: list[LLMMessage] = []
summaries: list[str] = []

for query in query_l:
    if len(history):
        summary_msg = await summarize_history(history)
        print(f"Summary: {summary_msg}")
        summaries.append(summary_msg)
        print("x" * 80)
        history_tmp = [LLMMessage(content=summary_msg, role=Roles.SYSTEM)]
    else:
        history_tmp = []

    response = await agent.answer(QueryWithContext(query=query, history=history_tmp))

    print("Answer:")
    print(f"{response.content}")
    print(f"\nSources ({len(response.sources)}):")
    for src in response.sources:
        source_file = src.metadata.get("source_file", "?")
        print(f"  {source_file!r}  |  {src.title!r}")

    history.append(LLMMessage(content=query, role=Roles.USER))
    history.append(LLMMessage(content=response.content, role=Roles.ASSISTANT))

    print("-" * 80)

Answer:
Based on the provided excerpts, I can confirm that PrimePack AG has the following pallets in its portfolio that have third-party verified Environmental Product Declarations (EPDs):

1. **Noé Pallet (32-100, CPR System)**
2. **Wooden Pallet 1208 (32-101, CPR System)**
3. **Recycled Plastic Pallet (32-102, CPR System)**
4. **Logypal 1 (32-103, Relicyc)**
5. **LogyLight (32-104, Relicyc)**
6. **EP 08 (32-105, StabilPlastik)**

The EPDs for these products are verified by accredited certification bodies, ensuring compliance with ISO 14025:2006 standards (source: excerpts from the documents). 

If you need further details about each specific pallet or the EPDs, please let me know!

Sources (5):
  'EPD_pallet_relicyc_logypal1.pdf'  |  '### THIRD-PARTY VERIFICATION'
  'ART_response_inquiry_frische_felder.md'  |  '### Response to Section 3: General'
  'EVALUATION_qa_ground_truth.md'  |  '## Q1: portfolio_scope'
  'EPD_tape_IPG_hotmelt.pdf'  |  '## **EPD Programme Information**'
  'EPD_t

In [10]:
history

[LLMMessage(content='Which pallets in our portfolio have a third-party verified EPD?', role=<Roles.USER: 'user'>, tool_calls=None, tool_call_id=None, name=None),
 LLMMessage(content='Based on the provided excerpts, I can confirm that PrimePack AG has the following pallets in its portfolio that have third-party verified Environmental Product Declarations (EPDs):\n\n1. **Noé Pallet (32-100, CPR System)**\n2. **Wooden Pallet 1208 (32-101, CPR System)**\n3. **Recycled Plastic Pallet (32-102, CPR System)**\n4. **Logypal 1 (32-103, Relicyc)**\n5. **LogyLight (32-104, Relicyc)**\n6. **EP 08 (32-105, StabilPlastik)**\n\nThe EPDs for these products are verified by accredited certification bodies, ensuring compliance with ISO 14025:2006 standards (source: excerpts from the documents). \n\nIf you need further details about each specific pallet or the EPDs, please let me know!', role=<Roles.ASSISTANT: 'assistant'>, tool_calls=None, tool_call_id=None, name=None),
 LLMMessage(content='Rewrite what y

In [11]:
summaries

['User inquired about which pallets in their portfolio have third-party verified Environmental Product Declarations (EPDs). Assistant provided a list of six pallets with verified EPDs from PrimePack AG, including details about each pallet and their compliance with ISO standards.',
 'The user inquired about which pallets in their portfolio have a third-party verified Environmental Product Declaration (EPD). The assistant provided a list of six pallets that meet this criterion, detailing their names and associated systems. The user then requested a concise, pirate-themed rewrite of the information.']

# Query reformulation

The query from the user might need to be aware of the history to search correctly in the vector store. Query reformulation asks an LLM to rephrase the query to make it more relevant for search. This was actually already done under the hood for you. 

See conversational_toolkit.utils.retriever -> make_query_standalone

In [12]:
logger.add(sys.stderr, level="DEBUG")

2

In [13]:
query_l = [
    "Hi, I'm Ivan! What is the pallet which has a 3rd-party verified EPD?",
    "What is the meaning of code?",
]

history: list[LLMMessage] = []

for query in query_l:
    response = await agent.answer(QueryWithContext(query=query, history=history))

    print("Answer:")
    print(f"{response.content}")
    print(f"\nSources ({len(response.sources)}):")
    for src in response.sources:
        source_file = src.metadata.get("source_file", "?")
        print(f"  {source_file!r}  |  {src.title!r}")

    history.append(LLMMessage(content=query, role=Roles.USER))
    history.append(LLMMessage(content=response.content, role=Roles.ASSISTANT))

    print("-" * 80)

2026-02-25 16:51:05.711 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:get_embeddings:76 - sentence-transformers/all-MiniLM-L6-v2 embeddings size: (1, 384)


Answer:
The excerpts provided do not specify which specific pallet has a third-party verified Environmental Product Declaration (EPD). They mention that the EPDs are verified by an accredited certification body, Bureau Veritas Italia S.p.A., and provide general information about the EPD system and its validity. However, there is no direct reference to a specific pallet or product associated with a third-party verified EPD.

If you have more specific information or additional excerpts that might indicate a particular pallet, please share them, and I can assist further. Otherwise, I do not know which pallet has a third-party verified EPD based on the provided excerpts.

Sources (5):
  'EPD_pallet_relicyc_logypal1.pdf'  |  '### THIRD-PARTY VERIFICATION'
  'EPD_pallet_relicyc_logypal1.pdf'  |  '# PROGRAMME INFORMATION'
  'EPD_tape_IPG_wateractivated.pdf'  |  '###### 1'
  'EPD_cardboard_redbox_cartonpallet.pdf'  |  '## **REDBOX Srl**'
  'ART_response_inquiry_frische_felder.md'  |  '### Resp

2026-02-25 16:51:09.571 | DEBUG    | conversational_toolkit.llms.openai:generate:87 - Completion: ChatCompletion(id='chatcmpl-DDBSG6woScEsSHpjcizH9gW2bsr4I', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='What is the meaning of code in relation to the pallet with a third-party verified EPD?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1772034668, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_373a14eb6f', usage=CompletionUsage(completion_tokens=19, prompt_tokens=316, total_tokens=335, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
2026-02-25 16:51:09.571 | INFO     | conversational_toolkit.llms.openai:generate:88 - LLM Usage: CompletionUs

Answer:
The code in relation to the pallet with a third-party verified Environmental Product Declaration (EPD) refers to the specific identification or classification of the product as per the Product Category Rules (PCR). The PCR outlines the requirements and guidelines for creating EPDs, ensuring that they conform to ISO 14025:2006 standards.

In the context of the excerpts provided, the EPDs must be verified by an accredited certification body, such as Bureau Veritas Italia S.p.A., and the EPD owner holds sole ownership and responsibility for the EPD. The environmental impacts of different EPDs can only be compared if they are based on the same declared/functional unit definition as outlined by the PCR (source: 9fe53c3c-32c8-4b4e-8f37-2ad9261fd30e).

Additionally, by December 31, 2025, all tier-1 suppliers are required to provide a valid third-party verified EPD for each product they supply, or a commitment letter with a credible EPD delivery schedule (source: 5ef9a5ca-def6-4e13-89b

---------------------------